### Import required modules

In [ ]:
import nltk
import random
import pickle
from nltk.classify import ClassifierI
from statistics import mode
from nltk.tokenize import word_tokenize
import os
import ipywidgets as widgets
from ipywidgets import Layout

### Load the previously trained classifier objects

In [ ]:
open_file = open("pickled_classifiers/NaiveBayes_classifier.pickle", "rb")
classifier = pickle.load(open_file)
open_file.close()

open_file = open("pickled_classifiers/MNB_classifier.pickle", "rb")
MNB_classifier = pickle.load(open_file)
open_file.close()

open_file = open("pickled_classifiers/BernoulliNB_classifier.pickle", "rb")
BernoulliNB_classifier = pickle.load(open_file)
open_file.close()

open_file = open("pickled_classifiers/LogisticRegression_classifier.pickle", "rb")
LogisticRegression_classifier = pickle.load(open_file)
open_file.close()

open_file = open("pickled_classifiers/LinearSVC_classifier.pickle", "rb")
LinearSVC_classifier = pickle.load(open_file)
open_file.close()

open_file = open("pickled_classifiers/SGDC_classifier.pickle", "rb")
SGDC_classifier = pickle.load(open_file)
open_file.close()

### Load BOW for featureset

In [ ]:
open_file = open("pickled_data/word_features.pickle", "rb")
word_features = pickle.load(open_file)
open_file.close()

### Define class for ensemble classifier i.e it will predict the sentiment based on multiple models

In [ ]:
class VoteClassifier(ClassifierI):
    def __init__(self, *classifiers):
        self._classifiers = classifiers

    def classify(self, features):
        votes = []
        for c in self._classifiers:
            v = c.classify(features)
            votes.append(v)
        return mode(votes)

    def confidence(self, features):
        votes = []
        for c in self._classifiers:
            v = c.classify(features)
            votes.append(v)

        choice_votes = votes.count(mode(votes))
        conf = choice_votes / len(votes)
        return conf

In [ ]:
def find_features(document):
    words = word_tokenize(document)
    features = {}
    for w in word_features:
        features[w] = (w in words)

    return features

In [ ]:
voted_classifier = VoteClassifier(classifier,LinearSVC_classifier,MNB_classifier,BernoulliNB_classifier,LogisticRegression_classifier)


def sentiment(text):
    feats = find_features(text)
    return [voted_classifier.classify(feats),voted_classifier.confidence(feats)]

In [ ]:
sentiment("Oh my god.")

### Define visual elements

In [ ]:
inputBox = widgets.Textarea(
    placeholder='Type the comment',
    disabled=False,
    layout=Layout(width='100%'))

button = widgets.Button(
    description='Click me',
    disabled=False,
    button_style='info', # 'success', 'info', 'warning', 'danger' or ''
    tooltip='Click me',
)

out = widgets.Output()

def on_button_clicked(b):
    inp = inputBox.value
    out.clear_output()
    if (inp != ''):
        res = sentiment(inp)
        predicted_sentiment = res[0]
        if (predicted_sentiment == 'pos'):
            predicted_sentiment = 'Positive'
        else:
            predicted_sentiment = 'Nagative'
        confidence_percent = res[1] * 100
        with out:
            print(str(int(confidence_percent)) + '% ' + predicted_sentiment + ' sentiment.')
    else:
        with out:
            print("Please type a comment in above box.")
        
button.on_click(on_button_clicked)

### Provide the input comment and click button for getting the prediction

In [ ]:
display(inputBox)
display(button)
display(out)